In [1]:
import os
import json

import os.path as osp

import pandas as pd 

from collections import Counter

import numpy as np

from loguru import logger


os.chdir(osp.dirname(os.getcwd()))





In [2]:
# from glob import glob
# from my_script.utils import readtxt
# consistency_list_qualitative_res_path = "/data3/xusc/exp/butd_detr/logs"

# all_data = {}

# for f in glob(consistency_list_qualitative_res_path+"/scanrefer20*" ):
#     data = np.loadtxt(f,delimiter='\n',dtype=np.str0).tolist()
#     name = f.split('/')[-1].split('.')[0]
#     print(len(data),name)
#     all_data[name] = data

In [3]:
# pick_samples = []
# for x in all_data['scanrefer20_all_consistenct_loss']:
#     if x not in  all_data['scanrefer20_no_constrastive_soft_token_consistency_loss_qualitative_list'] and \
#         x not in  all_data['scanrefer20_no_contrastive_consistency_loss_qualitative_list'] and \
#             x not in  all_data['scanrefer20_no_soft_token_consistency_loss_qualitative_list']:
#             pick_samples.append(x)

# len(pick_samples)
# np.savetxt(osp.join(consistency_list_qualitative_res_path,'only_all_consistency_works_qua_list.txt'),pick_samples,fmt="%s")

In [2]:



'''
description:  从SR3D作者那获取 详细的标注数据
param {*} split_name
return {*}
'''
def get_refer_it_3D(data_name,split=None):

    
    scanrefer_root="datasets/refer_it_3d"

    if split is None :
        data = pd.read_csv(osp.join(scanrefer_root,data_name+".csv"))
    else :
        data = pd.read_csv(osp.join(scanrefer_root,data_name+"_"+split+".csv"))

    # logger.info(f"len of {split_name} : {data.shape[0]}")
    # all_attrs = data.columns
    # logger.info(f" column of {split_name} : {all_attrs}")
    # logger.info(f"scene number : {len(set(data['scan_id']))}")
    # stat = Counter(data['scan_id'])
    # scane_stat = np.array([v for k ,v in stat.items()])
    # avg_sample =scane_stat.mean()
    # min_sample =scane_stat.min()
    # max_sample =scane_stat.max()

    # logger.info(f"min sample: {min_sample} \n max sample : {max_sample} \n avg sample: {avg_sample}")    
    # print(data.iloc[0,:])
    return data

def read_txt(file_name):
    with open(file_name,'r') as f:
        scan_ids = f.read().split("\n")
    logger.info(f" len of {file_name} : {len(scan_ids)}")
    return scan_ids

      

def save_txt(path,data):
    with open(path, 'w') as f:
        f.write(data)


In [14]:




class ClsEntity:

    def __init__(self,data_name, split,ratio=None):

        
        self.data_name=data_name
        self.split=split
        self.ratio=ratio

        scan_ids=None
        if ratio is None:
            scan_ids=self.get_split_list()
        else :
            scan_ids=self.get_ratio_split_list()

        self.scan_ids = scan_ids

        self.get_all_ann()

        

        
    '''
    description:  获取SR3D 作者划分好的 训练集 和测试集
    param {*} split
    return {*}
    '''
    def get_split_list(self):
        with open('data/meta_data/%s_%s_scans.txt' % (self.data_name,self.split)) as f:
            scan_ids = set(eval(f.read()))
        logger.info(f" length : {len(scan_ids)}")
        return scan_ids

        
        


    def get_ratio_split_list(self):
        with open('data/meta_data/%s_%s_%.1f.txt' % (self.data_name,self.split,self.ratio)) as f:
            scan_ids = f.read().split("\n")
        logger.info(f" length : {len(scan_ids)}")
        return scan_ids



    def get_all_ann(self):

      raise NotImplemented

        
        


class SR3DEntity(ClsEntity):


    def __init__(self,data_name, split,ratio=None):
        super().__init__(data_name, split,ratio)

    
    def get_all_ann(self):
        annos=  get_refer_it_3D(self.data_name,self.split)

        #* 根据当前scenario 进行过滤
        ans = []
        for idx in range(annos.shape[0]):

            ann = annos.iloc[idx]

            if ann.scan_id in self.scan_ids and ann['mentions_target_class'].lower() == 'true':
                ans.append(ann)
            
        self.annos = ans
        logger.info(f"annos num = {len(self.annos)}")


      



class NR3DEntity(ClsEntity):

    def __init__(self,data_name, split,ratio=None):
        super().__init__(data_name, split,ratio)
        

    '''
    description:  获取SR3D 作者划分好的 训练集 和测试集
    param {*} split
    return {*}
    '''
    def get_split_list(self):
        with open('data/meta_data/%s_%s_scans.txt' % (self.data_name,self.split)) as f:
            scan_ids = set(eval(f.read()))
        logger.info(f" length : {len(scan_ids)}")
        return scan_ids

       


    def get_ratio_split_list(self):
        with open('data/meta_data/%s_%s_%.1f.txt' % (self.data_name,self.split,self.ratio)) as f:
            scan_ids = f.read().split("\n")
        logger.info(f" assignment id length : {len(scan_ids)}")
        return np.array(scan_ids,dtype=np.int64).tolist()




    def get_all_ann(self):
        annos=  get_refer_it_3D(self.data_name)

        #* 根据当前scenario 进行过滤            
        ans = []
        for idx in range(annos.shape[0]):
            ann = annos.iloc[idx]
            #* 根据assignment id 来划分数据集
            if self.ratio is not None:
                if ann.assignmentid in self.scan_ids \
                    and str(ann['mentions_target_class']).lower() == 'true' \
                        and (str(ann['correct_guess']).lower() == 'true' or self.split != 'test'):
                    ans.append(ann)

            #* 根据 scan id 来划分数据集
            else:
                if ann.scan_id in self.scan_ids \
                    and str(ann['mentions_target_class']).lower() == 'true' \
                        and (str(ann['correct_guess']).lower() == 'true' or self.split != 'test'):
                    ans.append(ann)
                     
            
        self.annos = ans
        logger.info(f"annos num = {len(self.annos)}")

            

    def generate_NR3D_labeled_scene_txt(self,labeled_ratio):

        nr3d_ids = self.get_split_list(dataset_name='nr3d')

        num_scans = len(nr3d_ids)
        num_labeled_scans = int(num_scans*labeled_ratio)


        choices = np.random.choice(num_scans, num_labeled_scans, replace=False)#* 从num_scans 挑选num_labeled_scans 个场景 出来 

        labeled_scan_names = list(np.array(list(nr3d_ids))[choices])
        
        save_txt(os.path.join('data/meta_data/nr3d_train_{}.txt'.format(labeled_ratio)),'\n'.join(labeled_scan_names))
    
        logger.info('\tSelected {} labeled scans, remained {} unlabeled scans'.format(len(labeled_scan_names),num_scans- len(labeled_scan_names)))


        



    
    
# sr3d_ids = get_split_list()
# sr3d_ids_test = get_split_list(split = 'test')
# nr3d_ids = get_split_list(dataset_name='nr3d')
# nr3d_ids_test = get_split_list(dataset_name='nr3d',split = 'test')
# get_refer_it_3D('sr3d+')
# get_refer_it_3D('sr3d')
# get_refer_it_3D('sr3d_test')
# get_refer_it_3D('sr3d_train')



    

    

# a = ClsEntity('sr3d','train',0.2)
# a = ClsEntity('sr3d','test')
# a = NR3DEntity('nr3d','train',0.1)
# a = NR3DEntity('nr3d','train')
# a = NR3DEntity('nr3d','test')



for ratio in np.linspace(0.1,0.9,9):
    logger.info(ratio)
    NR3DEntity('nr3d','train',round(ratio,1))
    logger.info("==================================================================")



# for ratio in np.linspace(0.1,0.9,9):
#     logger.info(ratio)
#     SR3DEntity('nr3d','train',round(ratio,1))
#     logger.info("==================================================================")

2022-11-15 00:07:01.295 | INFO     | __main__:get_ratio_split_list:99 -  assignment id length : 3291


0.1


2022-11-15 00:07:23.296 | INFO     | __main__:get_all_ann:128 - annos num = 2884
2022-11-15 00:07:23.307 | INFO     | __main__:get_ratio_split_list:99 -  assignment id length : 6583


0.2


2022-11-15 00:07:57.690 | INFO     | __main__:get_all_ann:128 - annos num = 5498
2022-11-15 00:07:57.706 | INFO     | __main__:get_ratio_split_list:99 -  assignment id length : 9875


...

2022-11-15 00:13:27.283 | INFO     | __main__:get_all_ann:128 - annos num = 15122
2022-11-15 00:13:27.315 | INFO     | __main__:get_ratio_split_list:99 -  assignment id length : 26335


0.8


2022-11-15 00:14:59.546 | INFO     | __main__:get_all_ann:128 - annos num = 16470
2022-11-15 00:14:59.581 | INFO     | __main__:get_ratio_split_list:99 -  assignment id length : 29627


0.9


2022-11-15 00:16:38.988 | INFO     | __main__:get_all_ann:128 - annos num = 17926


In [6]:



def get_scene_data(all_data,scene = 'scene0525_00'):
    
    return_data = [] 
    for  idx in range(all_data.shape[0]):

        if all_data.iloc[idx]['scan_id'] == scene:
            return_data.append(all_data.iloc[idx])
            

    return return_data


def get_assignment_id(all_data,scene_list):
    assignments = []
    for idx in range(all_data.shape[0]):
        if all_data.iloc[idx]['scan_id'] in scene_list: 
            assignments.append (all_data.iloc[idx]['assignmentid'])
        
    return assignments




In [7]:

def split_labeled_according_assignment_id(assign_ids , ratio=0.4):
    
    length = len(assign_ids)
    choices = np.random.choice(length,int(length*ratio))

    return np.array(assign_ids)[choices]


def split_nr3d_according_to_assignmentid():
    for ratio in np.linspace(0.1,0.9,9):
        ratio = round(ratio,1)
        split_labeled_data = split_labeled_according_assignment_id(nr3d_all_assign_ids,ratio)
        print(f"length  = {len(split_labeled_data)}")
        save_txt(os.path.join('data/meta_data/nr3d_train_{}.txt'.format(ratio)),'\n'.join(split_labeled_data.astype(np.str0).tolist()))




def stat_mention_data_nr3d(nr3d_all_data,ratio=0.2):
    data = read_txt('data/meta_data/nr3d_train_{}.txt'.format(ratio))
    sum_=0
    for idx in range(nr3d_all_data.shape[0]):
        # if str(nr3d.iloc[idx]['assignmentid']) in data and nr3d.iloc[idx]['mentions_target_class']  and nr3d.iloc[idx]['correct_guess'] :
        if str(nr3d_all_data.iloc[idx]['assignmentid']) in data and nr3d_all_data.iloc[idx]['mentions_target_class']:
            sum_+=1
    logger.info(sum_)

# nr3d = get_refer_it_3D('nr3d')
# train_split = get_split_list(dataset_name='nr3d')
# nr3d_all_assign_ids = get_assignment_id(nr3d,train_split)
# save_txt(os.path.join('data/meta_data/nr3d_train_all_assignmentid.txt'),'\n'.join(np.array(nr3d_all_assign_ids).astype(np.str0).tolist()))


In [8]:

def get_scanrefer(split=None):
    if split is not None :
        path = "datasets/scanrefer/ScanRefer_filtered_%s.json"%split
    else :
        path = "datasets/scanrefer/ScanRefer_filtered.json"

    with open (path,'r')as f :
        data =json.load(f)
    
    length = len(data)
    logger.info(f" len of {split} split : {length}")
    # all_scene = set([x['scene_id']  for x in data])
    # logger.info(f" scene number  of {split} split : {len(all_scene)}")

    # all_object_id = set([x['object_id']  for x in data])
    # logger.info(f" object number  of {split} split : {len(all_object_id)}")

    # all_anno_id = set([x['ann_id']  for x in data])
    # logger.info(f" anno number  of {split} split : {len(all_anno_id)}")

    # print(data[0])
    
    return data

def get_ratio_scanrefer(ratio,split=None):
    
    path = "datasets/scanrefer/ScanRefer_filtered_train_%.1f.txt"%(ratio)

    scanrefer = get_scanrefer(split='train')
    scan_ids = read_txt(path)

    
    num = 0 
    for refer in scanrefer:
        if refer['scene_id'] in scan_ids:
            num+=1
    
    print(f"len of scanrefer {ratio} : {num}")
    



'''
description: 
param {*} labeled_ratio
return {*}
'''
def generate_scanrefer_labeled_scene_txt(labeled_ratio):
    all_scenes = get_scanrefer(split='train')

    num_scans = len(all_scenes)
    num_labeled_scans = int(num_scans*labeled_ratio)


    choices = np.random.choice(num_scans, num_labeled_scans, replace=False)#* 从num_scans 挑选num_labeled_scans 个场景 出来 

    labeled_scan_names = list(np.array(list(all_scenes))[choices])
    
    with open(os.path.join('datasets/scanrefer/ScanRefer_filtered_train_{}.txt'.format(labeled_ratio)), 'w') as f:
        f.write('\n'.join(labeled_scan_names))
    
    logger.info('\tSelected {} labeled scans, remained {} unlabeled scans'.format(len(labeled_scan_names),num_scans- len(labeled_scan_names)))


    

'''
description:  一次性生成所有比例的数据集划分
return {*}
'''
def generate_ratio_labeled_datasets():
    for x in np.linspace(0.1,0.9,9):
        generate_NR3D_labeled_scene_txt(round(x,1))
    for x in np.linspace(0.1,0.9,9):
        generate_scanrefer_labeled_scene_txt(round(x,1))

'''
description:  tmp code for stat 
return {*}
'''
def stat():
    datasets = ['sr3d','nr3d']

    for dataset in datasets:
        for n in np.linspace(0.1,0.9,9):
            # print(round(n,1))
            statstic_nr3d_by_ratio(round(n,1),dataset)
            
    for n in np.linspace(0.1,0.9,9):
        get_ratio_scanrefer(round(n,1))
        


# scan_val =get_scanrefer(split='val')


# get_scanrefer(split='test')
# get_scanrefer()
# scanrefer = get_scanrefer(split='train')






In [9]:





def get_scanrefer_data_by_scene(all_data,scene_name):
    
    res = [] 
    for line in all_data:
        # print(line['scene_id'])
        # print(line['object_id'])
        # print(line['object_name'])
        # print(line['ann_id'])

        # for k,v in line.items():
        #     print(k)
        if line['scene_id'] == scene_name:
            res.append(line)
    return res

scene_name = "scene0000_00"
scene_data = get_scanrefer_data_by_scene(scanrefer,scene_name)
print(f"len of scene : {len(scene_data)}")
print(scene_data[0])

NameError: name 'scanrefer' is not defined

In [ ]:




def stat_object_id_for_scene(scene_data,object_id):
    res = [] 
    for line in scene_data:
        if line['object_id'] == object_id:
            res.append(line)
    return res

def stat_ann_id_for_scene(scene_data,ann_id):
    res = [] 
    for line in scene_data:
        if line['ann_id'] == ann_id:
            res.append(line)
    return res

def stat_ann_id_and_object_id_for_scene(scene_data,ann_id,object_id):
    res = [] 
    for line in scene_data:
        if line['ann_id'] == ann_id and line['object_id'] == object_id:
            res.append(line)
    return res



# object_39 = stat_object_id_for_scene(scene_data,'39')
# ann_1=stat_ann_id_for_scene(scene_data,'1')
# object_39_ann_1 = stat_ann_id_and_object_id_for_scene(scene_data,'1','39')
# print(f"object_39 : {len(object_39)}")
# print(f"ann_1 : {len(ann_1)}")
# print(f"object_39_ann_1 : {len(object_39_ann_1)}")






len of scene : 173
{'scene_id': 'scene0000_00', 'object_id': '39', 'object_name': 'cabinet', 'ann_id': '1', 'description': 'a white cabinet in the corner of the room. in the direction from the door and from the inside . it will be on the left, there is a small brown table on the left side of the cabinet and a smaller table on the right side of the cabinet', 'token': ['a', 'white', 'cabinet', 'in', 'the', 'corner', 'of', 'the', 'room', '.', 'in', 'the', 'direction', 'from', 'the', 'door', 'and', 'from', 'the', 'inside', '.', 'it', 'will', 'be', 'on', 'the', 'left', ',', 'there', 'is', 'a', 'small', 'brown', 'table', 'on', 'the', 'left', 'side', 'of', 'the', 'cabinet', 'and', 'a', 'smaller', 'table', 'on', 'the', 'right', 'side', 'of', 'the', 'cabinet']}
object_39 : 5
ann_1 : 34
object_39_ann_1 : 1
